In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime
import time
import pickle
import gc
import warnings
import requests
import gzip
from io import BytesIO
from sklearn.preprocessing import StandardScaler


In [3]:
# Limpia la memoria RAM
gc.collect()

0

In [4]:
def leer_datos(ruta_archivo):
    # Asegúrate de que la ruta del archivo y el formato sean correctos
    df = pd.read_csv(ruta_archivo, compression='gzip', sep='\t')
    return df

In [5]:
file_url = 'https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_sellout_02.txt.gz'

# Read the file with a tab separator
df = pd.read_csv(file_url, sep='\t', compression='gzip')

In [6]:


# Create the DataFrame
df_sellout = pd.DataFrame(df)

stocks = leer_datos('https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_stocks_02.txt.gz')
master_prod =  pd.read_csv('https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_productos_02.txt', sep='\t')




In [7]:
df_sellout.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [7]:
precios_cui=df_sellout[['periodo','product_id','plan_precios_cuidados']].drop_duplicates().sort_values(by=['periodo','product_id'])
min_periods = df_sellout.groupby('product_id')['periodo'].min().reset_index()


In [8]:
# No hay stocks antes del periodo 201810
pd.DataFrame(stocks['periodo'].value_counts()).sort_values(by='periodo', ascending=False)


,count
periodo,
201912,925
201911,937
201910,949
201909,929
201908,928
201907,931
201906,914
201905,909
201904,915


In [9]:
# Convertir las columnas a string, int o float antes de hacer el merge
df_sellout['periodo'] = df_sellout['periodo'].astype(int)
df_sellout['product_id'] = df_sellout['product_id'].astype(int)
df_sellout['customer_id'] = df_sellout['customer_id'].astype(int)

df_sellout[['cust_request_qty', 'cust_request_tn','tn']] = df_sellout[['cust_request_qty', 'cust_request_tn','tn']].astype(float)


stocks['periodo'] = stocks['periodo'].astype(int)
stocks['product_id'] = stocks['product_id'].astype(int)
stocks['stock_final']=stocks['stock_final'].astype(float)

master_prod['product_id'] = master_prod['product_id'].astype(int)
master_prod['cat1'] = master_prod['cat1'].astype(str)
master_prod['cat2'] = master_prod['cat2'].astype(str)
master_prod['cat3'] = master_prod['cat3'].astype(str)
master_prod['brand'] = master_prod['brand'].astype(str)


master_prod['sku_size']=master_prod['sku_size'].astype(float)


In [10]:
df_sellout_prod=df_sellout.groupby(['periodo','product_id'])[['cust_request_qty', 'cust_request_tn','tn']].sum().reset_index()

In [11]:
df_sellout_prod_client=df_sellout.groupby(['periodo','customer_id','product_id'])[['cust_request_qty', 'cust_request_tn','tn']].sum().reset_index()

In [12]:
min_periods

,product_id,periodo
0,20001,201701
1,20002,201701
2,20003,201701
3,20004,201701
4,20005,201701
...,...,...
1228,21295,201701
1229,21296,201708
1230,21297,201701
1231,21298,201708


In [13]:
# Obtener la lista completa de periodos y product_id
all_periods = df['periodo'].unique()
all_product_ids = df['product_id'].unique()

# Crear un DataFrame con todas las combinaciones de customer_id, periodo y product_id
all_combinations = pd.MultiIndex.from_product([df['customer_id'].unique(), all_periods, all_product_ids], names=['customer_id', 'periodo', 'product_id'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

In [14]:
# Combinar el DataFrame completo con el DataFrame original para llenar con ceros en caso necesario
merged_df = pd.merge(all_combinations_df, df, on=['customer_id', 'periodo', 'product_id'], how='left')


In [15]:
precios_cui

,periodo,product_id,plan_precios_cuidados
44019,201701,20001,0
54801,201701,20002,0
13377,201701,20003,0
13202,201701,20004,0
13063,201701,20005,0
...,...,...,...
2937086,201912,21265,0
2937091,201912,21266,0
2936187,201912,21267,0
2944234,201912,21271,0


In [16]:
min_periods

,product_id,periodo
0,20001,201701
1,20002,201701
2,20003,201701
3,20004,201701
4,20005,201701
...,...,...
1228,21295,201701
1229,21296,201708
1230,21297,201701
1231,21298,201708


In [17]:
df = pd.merge(merged_df, min_periods, on='product_id', suffixes=('', '_min'))

In [18]:
df=pd.merge(df[['customer_id','periodo','product_id','cust_request_qty','cust_request_tn','tn','periodo_min']],precios_cui, on=['periodo','product_id'], how='left')
df.plan_precios_cuidados.fillna(0, inplace=True)

In [20]:
df

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados
0,10234,201701,20524,2.0,0.05300,0.05300,201701,0.0
1,10234,201702,20524,NaN,NaN,NaN,201701,0.0
2,10234,201703,20524,1.0,0.01514,0.01514,201701,0.0
3,10234,201704,20524,NaN,NaN,NaN,201701,0.0
4,10234,201705,20524,NaN,NaN,NaN,201701,0.0
...,...,...,...,...,...,...,...,...
26499631,10572,201908,20770,NaN,NaN,NaN,201912,0.0
26499632,10572,201909,20770,NaN,NaN,NaN,201912,0.0
26499633,10572,201910,20770,NaN,NaN,NaN,201912,0.0
26499634,10572,201911,20770,NaN,NaN,NaN,201912,0.0


In [19]:
def fillna_conditionally(df):
    mask = (df['periodo'] >= df['periodo_min']) & df['tn'].isnull()
    df['tn'] = df['tn'].where(~mask, 0)
    df['cust_request_qty'] = df['cust_request_qty'].where(~mask, 0)
    df['cust_request_tn'] = df['cust_request_tn'].where(~mask, 0)
    return df

In [20]:
fillna_conditionally(df)

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados
0,10234,201701,20524,2.0,0.05300,0.05300,201701,0.0
1,10234,201702,20524,0.0,0.00000,0.00000,201701,0.0
2,10234,201703,20524,1.0,0.01514,0.01514,201701,0.0
3,10234,201704,20524,0.0,0.00000,0.00000,201701,0.0
4,10234,201705,20524,0.0,0.00000,0.00000,201701,0.0
...,...,...,...,...,...,...,...,...
26499631,10572,201908,20770,NaN,NaN,NaN,201912,0.0
26499632,10572,201909,20770,NaN,NaN,NaN,201912,0.0
26499633,10572,201910,20770,NaN,NaN,NaN,201912,0.0
26499634,10572,201911,20770,NaN,NaN,NaN,201912,0.0


In [21]:
df

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados
0,10234,201701,20524,2.0,0.05300,0.05300,201701,0.0
1,10234,201702,20524,0.0,0.00000,0.00000,201701,0.0
2,10234,201703,20524,1.0,0.01514,0.01514,201701,0.0
3,10234,201704,20524,0.0,0.00000,0.00000,201701,0.0
4,10234,201705,20524,0.0,0.00000,0.00000,201701,0.0
...,...,...,...,...,...,...,...,...
26499631,10572,201908,20770,NaN,NaN,NaN,201912,0.0
26499632,10572,201909,20770,NaN,NaN,NaN,201912,0.0
26499633,10572,201910,20770,NaN,NaN,NaN,201912,0.0
26499634,10572,201911,20770,NaN,NaN,NaN,201912,0.0


,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados
0,10234,201701,20524,2.0,0.05300,0.05300,201701,0.0
1,10234,201702,20524,0.0,0.00000,0.00000,201701,0.0
2,10234,201703,20524,1.0,0.01514,0.01514,201701,0.0
3,10234,201704,20524,0.0,0.00000,0.00000,201701,0.0
4,10234,201705,20524,0.0,0.00000,0.00000,201701,0.0
...,...,...,...,...,...,...,...,...
26499631,10572,201908,20770,NaN,NaN,NaN,201912,0.0
26499632,10572,201909,20770,NaN,NaN,NaN,201912,0.0
26499633,10572,201910,20770,NaN,NaN,NaN,201912,0.0
26499634,10572,201911,20770,NaN,NaN,NaN,201912,0.0


In [22]:

df.sort_values(by=['customer_id','product_id','periodo' ])

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados
9028908,10001,201701,20001,11.0,99.43861,99.43861,201701,0.0
9028909,10001,201702,20001,23.0,198.84365,198.84365,201701,0.0
9028910,10001,201703,20001,33.0,92.46537,92.46537,201701,0.0
9028911,10001,201704,20001,8.0,13.29728,13.29728,201701,0.0
9028912,10001,201705,20001,15.0,101.20711,101.00563,201701,0.0
...,...,...,...,...,...,...,...,...
18738427,10637,201908,21299,0.0,0.00000,0.00000,201708,0.0
18738428,10637,201909,21299,0.0,0.00000,0.00000,201708,0.0
18738429,10637,201910,21299,0.0,0.00000,0.00000,201708,0.0
18738430,10637,201911,21299,0.0,0.00000,0.00000,201708,0.0


In [24]:
df_sellout_prod_client=df.sort_values(by=['customer_id','product_id','periodo' ])
df_sellout_prod_client['tn_mas_2']=df_sellout_prod_client.groupby(['customer_id','product_id'])['tn'].shift(-2)
df_sellout_prod_client=df_sellout_prod_client.reset_index(drop=True)


In [25]:
df_sellout_prod_client[((df_sellout_prod_client.tn_mas_2.isnull()) & (~df_sellout_prod_client.periodo.isin([201912,201911])))]

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados,tn_mas_2
1116,10001,201701,20032,NaN,NaN,NaN,201902,0.0,NaN
1117,10001,201702,20032,NaN,NaN,NaN,201902,0.0,NaN
1118,10001,201703,20032,NaN,NaN,NaN,201902,0.0,NaN
1119,10001,201704,20032,NaN,NaN,NaN,201902,0.0,NaN
1120,10001,201705,20032,NaN,NaN,NaN,201902,0.0,NaN
...,...,...,...,...,...,...,...,...,...
26499600,10637,201701,21299,NaN,NaN,NaN,201708,0.0,NaN
26499601,10637,201702,21299,NaN,NaN,NaN,201708,0.0,NaN
26499602,10637,201703,21299,NaN,NaN,NaN,201708,0.0,NaN
26499603,10637,201704,21299,NaN,NaN,NaN,201708,0.0,NaN


0

In [26]:

df_sellout_prod_client['tn_mas_2']=df_sellout_prod_client['tn_mas_2'].fillna(0)

In [27]:
df_sellout_prod_client.groupby(['customer_id','periodo'])['product_id'].count().reset_index()

,customer_id,periodo,product_id
0,10001,201701,1233
1,10001,201702,1233
2,10001,201703,1233
3,10001,201704,1233
4,10001,201705,1233
...,...,...,...
21487,10637,201908,1233
21488,10637,201909,1233
21489,10637,201910,1233
21490,10637,201911,1233


In [14]:
df_sellout_prod_client

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2
0,201701,10001,20001,11.0,99.43861,99.43861,198.84365,92.46537
1,201702,10001,20001,23.0,198.84365,198.84365,92.46537,13.29728
2,201703,10001,20001,33.0,92.46537,92.46537,13.29728,101.00563
3,201704,10001,20001,8.0,13.29728,13.29728,101.00563,128.04792
4,201705,10001,20001,15.0,101.20711,101.00563,128.04792,101.20711
...,...,...,...,...,...,...,...,...
2945813,201702,10636,20838,1.0,0.00190,0.00190,0.00000,0.00000
2945814,201702,10636,20969,1.0,0.00433,0.00433,0.00000,0.00000
2945815,201702,10636,21178,1.0,0.00197,0.00197,0.00000,0.00000
2945816,201709,10637,20657,1.0,0.00144,0.00144,0.00000,0.00000


# Vemos que el master tiene duplicados

In [28]:
duplicated_mask = master_prod.duplicated(subset=['product_id'], keep=False)
duplicated_cols = master_prod.loc[duplicated_mask, :]
duplicated_cols


,cat1,cat2,cat3,brand,sku_size,product_id
36,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20010
37,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20010
38,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20022
39,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20022
41,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20021
42,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20021
43,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20020
44,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20020
541,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI,2.0,20278
542,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI,2.0,20278


In [29]:
#eliminamos duplicados
master_prod.drop_duplicates(subset=['product_id'], inplace=True)


# Ahora mergeamos la base prod y la prod_customer

In [30]:
# Primero unimos df_sellout_prod con stocks
df_merge1 = pd.merge(df_sellout_prod_client, stocks, on=['periodo', 'product_id'], how='left')

# Luego unimos el resultado con master_prod
df_final_sellout_prod_client = pd.merge(df_merge1, master_prod, on='product_id', how='left')

df_final_sellout_prod_client['cat_1_2']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']
df_final_sellout_prod_client['cat_1_2_3']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']+'-'+df_final_sellout_prod_client['cat3']
df_final_sellout_prod_client['cat_1_2_3_brand']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']+'-'+df_final_sellout_prod_client['cat3']+'-'+df_final_sellout_prod_client['brand']

In [20]:
#df_test = df_final[(df_final['periodo'] >= 201810) & (df_final['periodo'] <= 201902)]
#df_test=df_test.reset_index(drop=True)


## Hasta aca tenemos dos bases, una que esta anivel de periodo y product_id y una que esta a nivel de periodo, product_id y customer_id.

In [38]:
# Periodo, cliente y producto
df_final_sellout_prod_client

,customer_id,periodo,product_id,cust_request_qty,cust_request_tn,tn,periodo_min,plan_precios_cuidados,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand
0,10001,201701,20001,11.0,99.43861,99.43861,201701,0.0,198.84365,92.46537,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
1,10001,201702,20001,23.0,198.84365,198.84365,201701,0.0,92.46537,13.29728,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
2,10001,201703,20001,33.0,92.46537,92.46537,201701,0.0,13.29728,101.00563,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
3,10001,201704,20001,8.0,13.29728,13.29728,201701,0.0,101.00563,128.04792,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
4,10001,201705,20001,15.0,101.20711,101.00563,201701,0.0,128.04792,101.20711,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26499631,10637,201908,21299,0.0,0.00000,0.00000,201708,0.0,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26499632,10637,201909,21299,0.0,0.00000,0.00000,201708,0.0,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26499633,10637,201910,21299,0.0,0.00000,0.00000,201708,0.0,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26499634,10637,201911,21299,0.0,0.00000,0.00000,201708,0.0,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Creamos una dummy para agosto 2019
df_final_sellout_prod_client['agosto_2019'] = np.where(df_final_sellout_prod_client['periodo'] == 201908, 1, 0)

In [32]:
# Creamos variable de mes y año
df_final_sellout_prod_client['year'] = df_final_sellout_prod_client['periodo'].astype(str).str[:4].astype(int)
df_final_sellout_prod_client['month'] = df_final_sellout_prod_client['periodo'].astype(str).str[4:].astype(int)

## División de train y test estrategia_a

In [33]:
mes_train=201811 # para entrenar se usa hasta el mes_train, el mes_train_post es para hacer luego el testeo que en t+2 es mes_test
mes_train_post=201812
mes_val=201901
mes_test=201902

In [34]:
df_final_sellout_prod_client_train=df_final_sellout_prod_client[(df_final_sellout_prod_client['periodo'] <= mes_train)].reset_index(drop=True)
df_final_sellout_prod_client_train_post=df_final_sellout_prod_client[(df_final_sellout_prod_client['periodo'] <= mes_train_post)].reset_index(drop=True)
df_final_sellout_prod_client_val=df_final_sellout_prod_client[(df_final_sellout_prod_client['periodo'] == mes_val)].reset_index(drop=True)
df_final_sellout_prod_client_test=df_final_sellout_prod_client[(df_final_sellout_prod_client['periodo'] == mes_test)].reset_index(drop=True)

### Probamos invertir la transformacion

### Estandarizamos cust_request_qty por product_id

### Probamos invirtiendo la estandarización

### Estandarizamos cust_request_tn por product_id

### Probamos invirtiendo la treandformacion

## Aca hacemos por categorias

### Para Train

In [34]:
cat_1=df_final_sellout_prod_client_train.groupby(['periodo','cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_tn'})
cat_2=df_final_sellout_prod_client_train.groupby(['periodo','cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_tn'})
cat_3=df_final_sellout_prod_client_train.groupby(['periodo','cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_tn'})
brand=df_final_sellout_prod_client_train.groupby(['periodo','brand'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_tn'})
cat_1_2=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_tn'})
cat_1_2_3=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2_3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_tn'})
cat_1_2_3_brand=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2_3_brand'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_tn'})


In [36]:
cat_1_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat1','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_cust_tn'})
cat_2_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_cust_tn'})
cat_3_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_cust_tn'})
brand_cust=df_final_sellout_prod_client_train.groupby(['periodo','brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_cust_tn'})
cat_1_2_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_cust_tn'})
cat_1_2_3_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2_3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_cust_tn'})
cat_1_2_3_brand_cust=df_final_sellout_prod_client_train.groupby(['periodo','cat_1_2_3_brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_cust_tn'})

In [37]:
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_cust, on=['periodo','customer_id', 'cat1'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_2_cust, on=['periodo', 'customer_id','cat2'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_3_cust, on=['periodo', 'customer_id','cat3'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(brand_cust, on=['periodo','customer_id', 'brand'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2_cust, on=['periodo','customer_id', 'cat_1_2'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2_3_cust, on=['periodo','customer_id', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2_3_brand_cust, on=['periodo','customer_id', 'cat_1_2_3_brand'], how='left')

In [38]:
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1, on=['periodo', 'cat1'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_2, on=['periodo','cat2'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_3, on=['periodo','cat3'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(brand, on=['periodo', 'brand'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2, on=['periodo', 'cat_1_2'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2_3, on=['periodo', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client_train = df_final_sellout_prod_client_train.merge(cat_1_2_3_brand, on=['periodo' ,'cat_1_2_3_brand'], how='left')

### Para Train_post


In [ ]:
cat_1=df_final_sellout_prod_client_train_post.groupby(['periodo','cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_tn'})
cat_2=df_final_sellout_prod_client_train_post.groupby(['periodo','cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_tn'})
cat_3=df_final_sellout_prod_client_train_post.groupby(['periodo','cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_tn'})
brand=df_final_sellout_prod_client_train_post.groupby(['periodo','brand'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_tn'})
cat_1_2=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_tn'})
cat_1_2_3=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2_3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_tn'})
cat_1_2_3_brand=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2_3_brand'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_tn'})


In [ ]:
cat_1_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat1','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_cust_tn'})
cat_2_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_cust_tn'})
cat_3_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_cust_tn'})
brand_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_cust_tn'})
cat_1_2_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_cust_tn'})
cat_1_2_3_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2_3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_cust_tn'})
cat_1_2_3_brand_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','cat_1_2_3_brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_cust_tn'})

In [ ]:
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_cust, on=['periodo','customer_id', 'cat1'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_2_cust, on=['periodo', 'customer_id','cat2'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_3_cust, on=['periodo', 'customer_id','cat3'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(brand_cust, on=['periodo','customer_id', 'brand'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2_cust, on=['periodo','customer_id', 'cat_1_2'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2_3_cust, on=['periodo','customer_id', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2_3_brand_cust, on=['periodo','customer_id', 'cat_1_2_3_brand'], how='left')

df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1, on=['periodo', 'cat1'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_2, on=['periodo','cat2'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_3, on=['periodo','cat3'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(brand, on=['periodo', 'brand'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2, on=['periodo', 'cat_1_2'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2_3, on=['periodo', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client_train_post = df_final_sellout_prod_client_train_post.merge(cat_1_2_3_brand, on=['periodo' ,'cat_1_2_3_brand'], how='left')

# Creamos variable de Producto menor tamaño y mayor tamaño en los que tienen presentaciones similares

### Train

In [45]:
prod_presen_cust=df_final_sellout_prod_client_train.groupby(['periodo','customer_id','cat_1_2_3_brand','sku_size'])['tn'].sum().reset_index()
prod_presen_cust=prod_presen_cust.sort_values(by=['cat_1_2_3_brand','periodo' ,'sku_size'])
# Sort the DataFrame by 'periodo' and 'cat_1_2_3_brand'
prod_presen_cust = prod_presen_cust.sort_values(by=['customer_id','periodo', 'cat_1_2_3_brand'])
# Create a column 'tn_smaller_sku' for the 'tn' value when 'sku_size' is smaller
prod_presen_cust['tn_smaller_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(1)

# Create a column 'tn_bigger_sku' for the 'tn' value when 'sku_size' is bigger
prod_presen_cust['tn_bigger_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(-1)

df_final_sellout_prod_client_train=pd.merge(df_final_sellout_prod_client_train, prod_presen_cust[['periodo','customer_id', 'cat_1_2_3_brand', 'sku_size', 'tn_smaller_sku_cust', 'tn_bigger_sku_cust']], on=['periodo','customer_id', 'cat_1_2_3_brand','sku_size'], how='left')



### Train_post

In [ ]:
prod_presen_cust=df_final_sellout_prod_client_train_post.groupby(['periodo','customer_id','cat_1_2_3_brand','sku_size'])['tn'].sum().reset_index()
prod_presen_cust=prod_presen_cust.sort_values(by=['cat_1_2_3_brand','periodo' ,'sku_size'])
# Sort the DataFrame by 'periodo' and 'cat_1_2_3_brand'
prod_presen_cust = prod_presen_cust.sort_values(by=['customer_id','periodo', 'cat_1_2_3_brand'])
# Create a column 'tn_smaller_sku' for the 'tn' value when 'sku_size' is smaller
prod_presen_cust['tn_smaller_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(1)

# Create a column 'tn_bigger_sku' for the 'tn' value when 'sku_size' is bigger
prod_presen_cust['tn_bigger_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(-1)

df_final_sellout_prod_client_train_post=pd.merge(df_final_sellout_prod_client_train_post, prod_presen_cust[['periodo','customer_id', 'cat_1_2_3_brand', 'sku_size', 'tn_smaller_sku_cust', 'tn_bigger_sku_cust']], on=['periodo','customer_id', 'cat_1_2_3_brand','sku_size'], how='left')



## Agregamos de fuentes externas el valor del dolar, ipc, TCRM, tasa de interes de caja de ahorro y plazo fijo y la temperatura maxima y minima para buenos aires

In [1]:
dolar=pd.read_csv('https://storage.googleapis.com/buko118/datasets/dolar_data.csv')
tempe=pd.read_csv('https://storage.googleapis.com/buko118/datasets/temperatura.csv')


dolar.periodo.astype(int)
tempe.periodo.astype(int)

NameError: name 'pd' is not defined

In [53]:
df_final_sellout_prod_client_train=pd.merge(df_final_sellout_prod_client_train, dolar, on='periodo', how='left')
df_final_sellout_prod_client_train_post=pd.merge(df_final_sellout_prod_client_train_post, dolar, on='periodo', how='left')

In [54]:
df_final_sellout_prod_client_train=pd.merge(df_final_sellout_prod_client_train, tempe, on='periodo', how='left')
df_final_sellout_prod_client_train_post=pd.merge(df_final_sellout_prod_client_train_post, tempe, on='periodo', how='left')

In [55]:
df_final_sellout_prod.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'tn_mas_1', 'tn_mas_2', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand',
       'sku_size', 'cat_1_2', 'cat_1_2_3', 'cat_1_2_3_brand', 'tn_scaled',
       'tn_mas_1_scaled', 'tn_mas_2_scaled', 'cust_request_qty_scaled',
       'cust_request_tn_scaled', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn',
       'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn',
       'stock_final_scaled', 'cat_1_tn_scaled', 'cat_2_tn_scaled',
       'cat_3_tn_scaled', 'brand_tn_scaled', 'cat_1_2_tn_scaled',
       'cat_1_2_3_tn_scaled', 'cat_1_2_3_brand_tn_scaled', 'tn_smaller_sku',
       'tn_bigger_sku', 'tn_smaller_sku_scaled', 'tn_bigger_sku_scaled',
       'usd_blue_max', 'usd_oficial_max', 'ipc', 'itcrm', 'int_ca', 'int_pf',
       'temp_max', 'temp_min'],
      dtype='object')

# Empezamos con el FE temporal 

## Considerando solo Product_id y periodo

El código proporcionado define una función llamada create_lagged_features que se utiliza para crear características de retraso, estadísticas móviles, características de tendencia y características de estacionariedad en un DataFrame de pandas. Estas características son útiles para modelar series temporales y otros tipos de datos secuenciales.

La función toma cuatro parámetros: df que es el DataFrame de entrada, lag_periods que es una lista de los períodos de retraso a considerar, window_sizes que es una lista de los tamaños de ventana para las estadísticas móviles, y feature_names que es una lista de los nombres de las características para las que se crearán las nuevas características.

Primero, la función ordena el DataFrame por 'product_id' y 'periodo'. Luego, para cada nombre de característica en feature_names, para cada tamaño de ventana en window_sizes, y para cada período de retraso en lag_periods, la función crea una nueva característica de retraso. Esto se hace utilizando el método shift de pandas, que desplaza los datos hacia abajo en un número especificado de filas (el período de retraso).

Después, para cada nombre de característica y cada tamaño de ventana, la función crea tres nuevas características: una media móvil, una desviación estándar móvil y una característica de tendencia. La media móvil y la desviación estándar móvil se calculan utilizando el método rolling de pandas, que proporciona una ventana móvil de los datos. La característica de tendencia se calcula utilizando el método diff de pandas, que calcula la diferencia entre filas consecutivas.

Finalmente, la función crea características de estacionariedad adicionales utilizando más características de retraso (hasta 24 períodos). Estas características se crean de la misma manera que las características de retraso anteriores.

La función devuelve el DataFrame con las características añadidas.

In [ ]:
import pandas as pd
import dask.dataframe as dd
from dask import delayed
from dask.diagnostics import ProgressBar

numeric_cols=[ 'cust_request_qty', 'cust_request_tn', 'tn','plan_precios_cuidados', 'stock_final', 'sku_size', 'cat_1_cust_tn', 'cat_2_cust_tn', 
              'cat_3_cust_tn', 'brand_cust_tn', 'cat_1_2_cust_tn', 'cat_1_2_3_cust_tn', 'cat_1_2_3_brand_cust_tn', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn', 
              'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn', 'tn_smaller_sku_cust', 'tn_bigger_sku_cust', 'usd_blue_max', 'usd_oficial_max',
              'ipc', 'itcrm', 'int_ca', 'int_pf', 'temp_max', 'temp_min']

def lagged_features_for_feature(df, feature_name, lag_periods, window_sizes):
    result_df = pd.DataFrame(index=df.index)

    for lag in lag_periods:
        result_df[f'{feature_name}_lag_{lag}'] = df.groupby(['customer_id', 'product_id'])[feature_name].shift(lag)
        result_df[f'{feature_name}_trend_lag_{lag}'] = df.groupby(['customer_id', 'product_id'])[feature_name].diff(lag)
        result_df[f'{feature_name}_lag_{lag}_ratio'] = df.groupby(['customer_id', 'product_id'])[feature_name].pct_change().shift(lag)
        result_df[f'{feature_name}_lag_{lag}_cumulative_sum'] = df.groupby(['customer_id', 'product_id'])[feature_name].cumsum().shift(lag)

    for window_size in window_sizes:
        result_df[f'{feature_name}_rolling_mean_win_{window_size}'] = df.groupby(['customer_id', 'product_id'])[feature_name].rolling(window=window_size).mean().reset_index(level=0, drop=True)
        result_df[f'{feature_name}_rolling_std_win_{window_size}'] = df.groupby(['customer_id', 'product_id'])[feature_name].rolling(window=window_size).std().reset_index(level=0, drop=True)

    return result_df

def create_lagged_features(df, lag_periods, window_sizes, feature_names):
    df = df.sort_values(by=['customer_id','product_id', 'periodo'])
    delayed_results = []

    for feature_name in feature_names:
        delayed_results.append(
            delayed(lagged_features_for_feature)(df, feature_name, lag_periods, window_sizes)
        )

    with ProgressBar():
        lagged_dfs = dd.compute(*delayed_results, num_workers=16)

    # Concatenate the lagged DataFrames for each feature
    df_final = pd.concat(lagged_dfs, axis=1)

    return df_final

# Example usage
lag_periods = range(1, 13)  # Consider lags from 1 to 12 periods
window_sizes = range(1, 13)  # Consider window sizes from 1 to 12

# Assuming df_sellout_prod_train is your pandas DataFrame
df_final_sellout_prod_client_train_fe = create_lagged_features(df_final_sellout_prod_client_train, lag_periods, window_sizes, numeric_cols)

ddf_final_sellout_prod_client_train_fe=df_final_sellout_prod_client_train.join(ddf_final_sellout_prod_client_train_fe)

In [ ]:
import pandas as pd
import dask.dataframe as dd
from dask import delayed
from dask.diagnostics import ProgressBar

numeric_cols=[ 'cust_request_qty', 'cust_request_tn', 'tn','plan_precios_cuidados', 'stock_final', 'sku_size', 'cat_1_cust_tn', 'cat_2_cust_tn', 
              'cat_3_cust_tn', 'brand_cust_tn', 'cat_1_2_cust_tn', 'cat_1_2_3_cust_tn', 'cat_1_2_3_brand_cust_tn', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn', 
              'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn', 'tn_smaller_sku_cust', 'tn_bigger_sku_cust', 'usd_blue_max', 'usd_oficial_max',
              'ipc', 'itcrm', 'int_ca', 'int_pf', 'temp_max', 'temp_min']

def lagged_features_for_feature(df, feature_name, lag_periods, window_sizes):
    result_df = pd.DataFrame(index=df.index)

    for lag in lag_periods:
        result_df[f'{feature_name}_lag_{lag}'] = df.groupby(['customer_id', 'product_id'])[feature_name].shift(lag)
        result_df[f'{feature_name}_trend_lag_{lag}'] = df.groupby(['customer_id', 'product_id'])[feature_name].diff(lag)
        result_df[f'{feature_name}_lag_{lag}_ratio'] = df.groupby(['customer_id', 'product_id'])[feature_name].pct_change().shift(lag)
        result_df[f'{feature_name}_lag_{lag}_cumulative_sum'] = df.groupby(['customer_id', 'product_id'])[feature_name].cumsum().shift(lag)

    for window_size in window_sizes:
        result_df[f'{feature_name}_rolling_mean_win_{window_size}'] = df.groupby(['customer_id', 'product_id'])[feature_name].rolling(window=window_size).mean().reset_index(level=0, drop=True)
        result_df[f'{feature_name}_rolling_std_win_{window_size}'] = df.groupby(['customer_id', 'product_id'])[feature_name].rolling(window=window_size).std().reset_index(level=0, drop=True)

    return result_df

def create_lagged_features(df, lag_periods, window_sizes, feature_names):
    df = df.sort_values(by=['customer_id','product_id', 'periodo'])
    delayed_results = []

    for feature_name in feature_names:
        delayed_results.append(
            delayed(lagged_features_for_feature)(df, feature_name, lag_periods, window_sizes)
        )

    with ProgressBar():
        lagged_dfs = dd.compute(*delayed_results, num_workers=16)

    # Concatenate the lagged DataFrames for each feature
    df_final = pd.concat(lagged_dfs, axis=1)

    return df_final

# Example usage
lag_periods = range(1, 13)  # Consider lags from 1 to 12 periods
window_sizes = range(1, 13)  # Consider window sizes from 1 to 12

# Assuming df_sellout_prod_train is your pandas DataFrame
df_final_sellout_prod_client_train_post_fe = create_lagged_features(df_final_sellout_prod_client_train_post, lag_periods, window_sizes, numeric_cols)

df_final_sellout_prod_client_train_post_fe=df_final_sellout_prod_client_train_post.join(df_final_sellout_prod_client_train_post_fe)

In [ ]:
#Agregamos el indice para poder coordinar posteriormente con el scaler
df_final_sellout_prod_client_train_fe['id_row']=df_final_sellout_prod_client_train_fe.index
df_final_sellout_prod_client_train_post_fe['id_row']=df_final_sellout_prod_client_train_post_fe.index

columns_order = ['id_row'] + [col for col in df_final_sellout_prod_client_train_fe.columns if col != 'id_row']
df_final_sellout_prod_client_train_fe = df_final_sellout_prod_client_train_fe[columns_order]

columns_order = ['id_row'] + [col for col in df_final_sellout_prod_client_train_post_fe.columns if col != 'id_row']
df_final_sellout_prod_client_train_post_fe = df_final_sellout_prod_client_train_post_fe[columns_order]

In [ ]:
df_final_sellout_prod_client_train_fe.to_parquet('/home/all4data118/buckets/b1/datasets/df_final_sellout_prod_client_train_fe_es_a.parquet', index=False)
df_final_sellout_prod_client_train_post_fe.to_parquet('/home/all4data118/buckets/b1/datasets/df_final_sellout_prod_client_train_post_fe_es_a.parquet', index=False)

df_final_sellout_prod_client_val.to_parquet('/home/all4data118/buckets/b1/datasets/df_final_sellout_prod_client_val_es_a.parquet', index=False)
df_final_sellout_prod_client_test.to_parquet('/home/all4data118/buckets/b1/datasets/df_final_sellout_prod_client_val_es_a.parquet', index=False)